## 3.1 NLP extract ingredients

Preprocess Reviews.txt

In [11]:
#clean Reviews.txt
import pandas as pd
import re

# Read the entire file as raw text
with open('Reviews_test.txt', 'r', encoding='utf-8') as file:
    raw_text = file.read()

# Replace triple quotes and newlines within reviews
# Keep newlines that separate proper reviews (i.e., before ReviewId)
cleaned_text = re.sub(r'(""""|\n)(?!(\d+\t))', ' ', raw_text)

# Now cleaned_text should have each review on a single line, except for proper row separations
from io import StringIO

# Use StringIO to treat the cleaned text like a file
cleaned_file = StringIO(cleaned_text)

# Load into pandas DataFrame
rev_df = pd.read_csv(cleaned_file, delimiter='\t', on_bad_lines='skip')

In [ ]:
# #### With csv for testing
# import pandas as pd
# from collections import defaultdict
# from datetime import datetime
# from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
# import csv

# # rev_df = pd.read_csv('path_to_your_cleaned_file.csv')  # Uncomment if you need to reload

# # NER model for food
# tokenizer_ner = AutoTokenizer.from_pretrained("Dizex/InstaFoodRoBERTa-NER")
# model_ner = AutoModelForTokenClassification.from_pretrained("Dizex/InstaFoodRoBERTa-NER").to("cuda")
# ner_pipe = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner, device=0) # device=-1 for CPU

# # Function to convert entities to a list
# def convert_entities_to_list(text, entities):
#     ents = []
#     for ent in entities:
#         e = {"start": ent["start"], "end": ent["end"], "label": ent["entity_group"]}
#         if ents and -1 <= ent["start"] - ents[-1]["end"] <= 1 and ents[-1]["label"] == e["label"]:
#             ents[-1]["end"] = e["end"]
#             continue
#         ents.append(e)
#     return [text[e["start"]:e["end"]] for e in ents]

# # Initialize storage for results
# results = []
# ingredient_counts = defaultdict(int)

# # Process each review
# for idx, row in rev_df.iterrows():
#     review_text = row.get('Review', '')  # Ensure the column name matches your DataFrame
    
#     # Run NER model
#     ner_entity_results = ner_pipe(review_text, aggregation_strategy="simple")
#     ingredients = convert_entities_to_list(review_text, ner_entity_results)

#     # Update ingredient counts
#     for ingredient in ingredients:
#         ingredient_counts[ingredient] += 1

#     # Store the results
#     results.append({
#         'Review': review_text,
#         'Extracted Ingredients': ', '.join(ingredients) if ingredients else 'None'
#     })

# # Generate timestamps for unique filenames
# current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
# extracted_file = f"{current_time}_extracted_ingredients.csv"
# counted_file = f"{current_time}_ingredient_counts.csv"


# # Write extracted ingredients to a CSV file
# with open(extracted_file, 'w', newline='', encoding='utf-8') as csvfile:
#     fieldnames = ['Review', 'Extracted Ingredients']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#     writer.writeheader()
#     writer.writerows(results)

# print(f"Extracted ingredients saved to {extracted_file}")

# # Write ingredient counts to a separate CSV file
# with open(counted_file, 'w', newline='', encoding='utf-8') as csvfile:
#     fieldnames = ['Ingredient', 'Count']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#     writer.writeheader()
#     for ingredient, count in sorted(ingredient_counts.items(), key=lambda x: x[1], reverse=True):
#         writer.writerow({'Ingredient': ingredient, 'Count': count})

# print(f"Ingredient counts saved to {counted_file}")


c:\Users\Gebruiker\Desktop\school\cursuri\cuda_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Extracted ingredients saved to 20250213_181954_extracted_ingredients.csv
Ingredient counts saved to 20250213_181954_ingredient_counts.csv


Use NLP to extract ingredients

In [12]:
#### Without csv
import pandas as pd
from collections import defaultdict
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import csv

# Load your DataFrame (assuming 'rev_df' is already prepared)
# rev_df = pd.read_csv('path_to_your_cleaned_file.csv')  # Uncomment if needed

# NER model for food
tokenizer_ner = AutoTokenizer.from_pretrained("Dizex/InstaFoodRoBERTa-NER")
model_ner = AutoModelForTokenClassification.from_pretrained("Dizex/InstaFoodRoBERTa-NER").to("cuda")
ner_pipe = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner, device=0)  # device=-1 for CPU

# Function to convert entities to a list
def convert_entities_to_list(text, entities):
    ents = []
    for ent in entities:
        e = {"start": ent["start"], "end": ent["end"], "label": ent["entity_group"]}
        if ents and -1 <= ent["start"] - ents[-1]["end"] <= 1 and ents[-1]["label"] == e["label"]:
            ents[-1]["end"] = e["end"]
            continue
        ents.append(e)
    return [text[e["start"]:e["end"]] for e in ents]

# Initialize storage for ingredient counts
ingredient_counts = defaultdict(int)
ingredients_list = []  # Store extracted ingredients for DataFrame

# Process each review
for idx, row in rev_df.iterrows():
    review_text = row.get('Review', '')  # Ensure the column name matches your DataFrame

    # Run NER model
    ner_entity_results = ner_pipe(review_text, aggregation_strategy="simple")
    ingredients = convert_entities_to_list(review_text, ner_entity_results)

    # Update ingredient counts
    for ingredient in ingredients:
        ingredient_counts[ingredient] += 1

    # Store extracted ingredients for DataFrame
    ingredients_list.append(', '.join(ingredients) if ingredients else 'None')

# Add new "Ingredients" column to rev_df
rev_df['Ingredients'] = ingredients_list

# Generate timestamps for unique filenames
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
extracted_file = f"{current_time}_extracted_ingredients.csv"
counted_file = f"{current_time}_ingredient_counts.csv"

# Save updated DataFrame to CSV (optional)
rev_df.to_csv(extracted_file, index=False, encoding='utf-8')
print(f"Updated DataFrame with extracted ingredients saved to {extracted_file}")

# Write ingredient counts to a separate CSV file
with open(counted_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Ingredient', 'Count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for ingredient, count in sorted(ingredient_counts.items(), key=lambda x: x[1], reverse=True):
        writer.writerow({'Ingredient': ingredient, 'Count': count})

print(f"Ingredient counts saved to {counted_file}")


Device set to use cuda:0


Updated DataFrame with extracted ingredients saved to 20250213_230834_extracted_ingredients.csv
Ingredient counts saved to 20250213_230834_ingredient_counts.csv


Add extracted ingredients to our KG

In [ ]:
## Add ingredients to Review class
# Add Review Properties for extracted ingredients
extr_ingred = foodnutrition["extractedIngredients"]

g.add((extr_ingred, RDFS.domain, SDO.Review))
g.add((extr_ingred, RDFS.range, RDFS.Literal))


for idx, row in rev_df.iterrows():
    review_uri = foodnutrition[f"review_{row['ReviewId']}"]  # Unique review identifier
    g.add((review_uri, RDF.type, SDO.Review))  # Add review instance

    # Iterate over individual ingredients and add separately
    for ingredient in row["Ingredients"].split(", "):  # Assuming ingredients are comma-separated
        g.add((review_uri, extr_ingred, Literal(ingredient)))  # Add each ingredient as a separate triple


# Save the updated KG to a file (optional)
g.serialize("updated_kg.ttl", format="turtle")

print("Ingredients added to the Knowledge Graph!")


Ingredients successfully added to the Knowledge Graph!
